In [316]:
# import xml.etree.ElementTree as ET
# import json

# def extract_metadata(emxml_file, json_file):
#     # Load JSON file
#     with open(json_file, 'r') as f:
#         metadata_mapping = json.load(f)

#     # Load XML file
#     tree = ET.parse(emxml_file)
#     root = tree.getroot()

#     metadata = {}

#     # Extract acquisition metadata
#     acquisition_metadata = metadata_mapping.get('acquisition')
#     for key, value in acquisition_metadata.items():
#         node = root.find(value)
#         if node is not None:
#             metadata[key] = node.text

#     # Extract dataset metadata
#     dataset_metadata = metadata_mapping.get('dataset')
#     for key, value in dataset_metadata.items():
#         node = root.find(value)
#         if node is not None:
#             metadata[key] = node.text

#     # Extract image metadata
#     image_metadata = metadata_mapping.get('image')
#     for key, value in image_metadata.items():
#         node = root.find(value)
#         if node is not None:
#             metadata[key] = node.text

#     return metadata

# # Usage example
# json_file = '/Users/elias/Desktop/PP13_Mapping/pp13-mapper/schemas/sem_fib_nested_schema_map.json'
# emxml_file = '/Users/elias/Desktop/NFDI Tomographiedaten/20200818_AlSi13 XRM tomo2/EMproject.emxml'
# metadata = extract_metadata(emxml_file, json_file)
# print(metadata)


In [323]:
import json

def extract_EM_metadata(json_file):
    # Read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Extract the "acquisition" information
    acquisition_data = data.get('acquisition', {})

    # Store key-value pairs starting with "EMproject"
    EM_metadata = {}
    for key, value in acquisition_data.items():
        if value.startswith('EMProject'):
            EM_metadata[key] = value

    return EM_metadata

emMetadata = extract_EM_metadata(json_file)

# print(type(emMetadata))


# for key, value in emMetadata.items():
#     print(key + ": " + value)

In [325]:
import xml.etree.ElementTree as ET

def xml_to_dict(file_path):
    def parse_element(element):
        result = {}
        if len(element) == 0:
            return element.text
        for child in element:
            child_data = parse_element(child)
            if '}' in child.tag:
                child_tag = child.tag.split('}', 1)[1]  # Remove the namespace
            else:
                child_tag = child.tag
            if child_tag in result:
                if type(result[child_tag]) is list:
                    result[child_tag].append(child_data)
                else:
                    result[child_tag] = [result[child_tag], child_data]
            else:
                result[child_tag] = child_data
        return result

    tree = ET.parse(file_path)
    root = tree.getroot()
    root_tag = root.tag
    if '}' in root_tag:
        root_tag = root_tag.split('}', 1)[1]  # Remove the namespace from the root tag
    return {root_tag: parse_element(root)}

In [326]:
emData = xml_to_dict(emxml_file)

In [327]:
def traverse_dict(data, path):
    keys = path.split('.')
    result = data
    try:
        for key in keys:
            result = result[key]
        return result
    except (KeyError, TypeError):
        return None

In [328]:
def extract_values(addresses, data, dataset_num = 1):
    result = {}
    for key, address in addresses.items():
        levels = address.split('.')
        current_data = data
        for level in levels:
            if level == 'Dataset':
                current_data = current_data[level][dataset_num - 1]
            else:
                current_data = current_data[level]
        result[key] = current_data
    return result

In [329]:
extract_values(emMetadata, emData)

{'acquisition.genericMetadata.program.programName': 'Auto Slice & View 4',
 'acquisition.genericMetadata.program.programVersion': '4.2.1.1982',
 'acquisition.genericMetadata.applicationId.identifierValue': 'ASV',
 'acquisition.genericMetadata.fileVersion': '1.2',
 'acquisition.genericMetadata.projectName': '20200818_AlSi13 XRM tomo2',
 'acquisition.genericMetadata.zCutSpacing.value': '2.0000000000000002E-07',
 'acquisition.genericMetadata.numberOfCuts': '719'}